In [1]:
# import Pkg; Pkg.add("Tables"); Pkg.add("MarkdownTables"); Pkg.add("DataFrames")

In [2]:
# using ProgressMeter
include("datasetloader.jl")
include("benchmark.jl")

showtable (generic function with 2 methods)

In [3]:
WV = WikiDataSet("corpus/wikipedia/test", langs=LANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [4]:
# import Pkg; Pkg.add(["LanguageFinder", "LanguageDetect", "Languages"])

In [5]:
# import LanguageFinder: LanguageFind
# detector_lf(t) = LanguageFind(t, 0).lang
import LanguageDetect: detect
detector_ld(t) = first(detect(t)).language
import Languages: LanguageDetector, isocode
Languages_detector = LanguageDetector()
function detector_ls(t)
    l = Languages_detector(t) |> first
    if l == nothing
        return ""
    else
        return l |> isocode
    end
end
using Logging
Logging.disable_logging(Logging.Info)

LogLevel(1)

In [6]:
include("detector.jl")
detector_lh

detector_lh (generic function with 1 method)

## tatoeba

In [7]:
TB = benchmark(
"detector_ngram1"=>t->detector_lh(t, ngram=1),
"detector_ngram2"=>t->detector_lh(t, ngram=2),
"detector_ngram3"=>t->detector_lh(t, ngram=3),
"detector_ngram4"=>t->detector_lh(t, ngram=4),
"detector_ngram5"=>detector_lh,
"Languages.jl"=>detector_ls, 
"LanguageDetect.jl"=>detector_ld,
 dataset=TV, languages=LANGS)

detector_ngram1 44.642954 seconds (40.47 M allocations: 3.161 GiB, 1.36% gc time, 0.63% compilation time)
detector_ngram2

147.257007 seconds (63.23 M allocations: 5.854 GiB, 0.73% gc time, 0.05% compilation time)
detector_ngram3

233.136634 seconds (85.08 M allocations: 8.258 GiB, 0.61% gc time, 0.07% compilation time)
detector_ngram4

326.500113 seconds (106.08 M allocations: 10.882 GiB, 0.54% gc time, 0.03% compilation time)


detector_ngram5434.691679 seconds (126.62 M allocations: 14.058 GiB, 0.47% gc time, 0.01% compilation time)
Languages.jl

296.589494 seconds (144.15 M allocations: 8.056 GiB, 0.43% gc time, 0.91% compilation time)
LanguageDetect.jl

2017.803291 seconds (10.89 G allocations: 356.530 GiB, 2.84% gc time, 0.09% compilation time)


7-element Vector{Any}:
   "detector_ngram1" => [0.9936134453781512, 0.9662261380323054, 1.0, 0.7371382636655949, 0.4230769230769231, 0.7125962596259626, 0.5125386465132257, 0.6360044965472941, 1.0, 0.745218519169891  …  0.24251873459739476, 0.8573825503355704, 0.6934140802422407, 0.8863955119214586, 0.7975935828877005, 0.8651508887970235, 0.8091818973020017, 0.9974704890387859, 0.9517241379310345, 1.0]
   "detector_ngram2" => [0.9919327731092437, 0.9632892804698973, 1.0, 0.7813504823151125, 0.78125, 0.8721122112211221, 0.7124699416008244, 0.9166211658904769, 1.0, 0.9390828350334154  …  0.7356535734044158, 0.9496644295302014, 0.8705526116578349, 0.9032258064516129, 0.9483957219251337, 0.9612787653300262, 0.8046127067014796, 0.9974704890387859, 0.9793103448275862, 1.0]
   "detector_ngram3" => [0.9909243697478992, 0.9720998531571219, 1.0, 0.8110932475884244, 0.8990384615384616, 0.9251925192519251, 0.8217107523187908, 0.9783523365986831, 1.0, 0.9835011431586352  …  0.9001156767087461, 0.97

In [8]:
showtable(TB, LANGS)

|                   | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-------------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  74.

|                   | Average | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit    | mar    | mkd    | msa    | nds    | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat    | tgl    | tur    | ukr    | vie    | yid    | zho     |
|-------------------|---------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  74.21% | 99.36% | 96.62% | 100.00% | 73.71% | 42.31% | 71.26% | 51.25% | 63.60% | 100.00% | 74.52% | 63.50% | 97.13% | 88.35% | 48.13% | 60.22% |  8.56% |  98.44% | 80.18% | 88.91% | 28.52% |  5.18% | 90.95% | 38.29% | 99.87% | 93.23% | 100.00% | 98.56% | 84.80% | 92.87% | 83.30% | 80.28% | 64.03% | 41.25% | 62.82% | 38.70% | 96.67% | 59.21% | 84.18% | 78.59% | 73.26% | 24.25% | 85.74% | 69.34% | 88.64% | 79.76% | 86.52% | 80.92% | 99.75% | 95.17% | 100.00% |
|   detector_ngram2 |  88.88% | 99.19% | 96.33% | 100.00% | 78.14% | 78.12% | 87.21% | 71.25% | 91.66% | 100.00% | 93.91% | 87.60% | 98.09% | 97.88% | 88.50% | 95.23% |  9.44% |  98.31% | 84.65% | 97.18% | 76.55% | 66.41% | 97.29% | 87.28% | 99.95% | 98.77% | 100.00% | 99.36% | 93.34% | 98.15% | 85.64% | 86.50% | 91.80% | 81.30% | 92.07% | 71.96% | 98.90% | 89.80% | 91.21% | 84.25% | 85.94% | 73.57% | 94.97% | 87.06% | 90.32% | 94.84% | 96.13% | 80.46% | 99.75% | 97.93% | 100.00% |
|   detector_ngram3 |  93.20% | 99.09% | 97.21% | 100.00% | 81.11% | 89.90% | 92.52% | 82.17% | 97.84% | 100.00% | 98.35% | 96.84% | 98.60% | 99.02% | 96.67% | 98.08% | 11.19% |  98.61% | 85.04% | 98.83% | 88.74% | 88.74% | 98.64% | 96.36% | 99.97% | 99.47% | 100.00% | 99.52% | 95.91% | 99.31% | 85.18% | 91.75% | 96.34% | 93.84% | 96.77% | 81.11% | 99.29% | 96.53% | 95.79% | 89.77% | 90.63% | 90.01% | 97.90% | 93.91% | 91.58% | 98.02% | 98.64% | 86.10% | 99.92% | 98.97% |  99.97% |
|   detector_ngram4 |  94.59% | 98.92% | 97.65% | 100.00% | 85.05% | 92.79% | 94.39% | 86.71% | 99.07% | 100.00% | 99.30% | 98.89% | 98.97% | 99.28% | 98.30% | 98.72% | 16.04% |  99.01% | 88.06% | 99.19% | 90.24% | 94.82% | 99.25% | 97.96% | 99.97% | 99.57% | 100.00% | 99.68% | 96.08% | 99.53% | 86.57% | 93.80% | 97.69% | 96.48% | 97.83% | 83.78% | 99.42% | 98.14% | 97.11% | 92.13% | 91.77% | 94.15% | 98.49% | 95.99% | 92.15% | 99.01% | 99.11% | 89.35% | 99.92% | 99.20% |  99.97% |
|   detector_ngram5 |  95.04% | 99.09% | 97.65% | 100.00% | 87.06% | 93.27% | 94.64% | 87.60% | 99.27% | 100.00% | 99.37% | 99.03% | 99.27% | 99.21% | 98.52% | 99.08% | 21.89% |  99.14% | 91.34% | 99.21% | 91.37% | 95.65% | 99.25% | 98.00% | 99.97% | 99.55% | 100.00% | 99.68% | 95.67% | 99.57% | 87.36% | 94.70% | 97.97% | 96.92% | 97.97% | 83.55% | 99.49% | 98.23% | 97.24% | 93.16% | 91.89% | 94.61% | 98.57% | 96.25% | 92.15% | 99.12% | 99.13% | 90.97% | 99.92% | 99.43% |  99.97% |
|      Languages.jl |  57.66% | 85.55% | 80.47% | 100.00% | 62.46% |      - | 48.93% | 47.06% | 90.48% |  99.89% | 78.21% | 64.61% | 95.00% | 76.87% | 82.21% | 92.85% | 60.28% |  95.75% | 62.99% | 74.00% |      - |      - |      - | 66.27% | 69.37% |      - |  98.97% |      - |      - | 61.94% | 72.05% | 51.40% | 71.26% |      - | 78.91% | 66.74% | 72.74% | 77.35% | 70.87% | 52.59% |      - | 61.89% |      - | 52.46% |      - | 63.98% | 52.10% | 62.63% | 84.06% | 98.39% |  99.51% |
| LanguageDetect.jl |  60.42% | 93.61% |      - | 100.00% | 64.47% | 56.25% | 71.64% | 53.66% | 82.15% | 100.00% | 74.55% |      - | 93.53% | 90.31% | 76.88% |      - | 25.98% | 100.00% | 91.60% | 86.49% |      - |      - |      - | 69.26% | 99.84% |      - |  99.48% |      - |      - | 80.60% | 86.74% | 74.55% | 85.82% |      - | 65.21% | 52.72% | 92.47% | 70.31% | 83.49% | 78.27% | 55.43% | 60.45% | 83.89% | 70.36% |      - | 90.83% | 90.27% | 71.78% | 99.49% |      - |  98.45% |


In [17]:
showtable(TB, LANGS, threshold=0.)

50 columns in total, 35 columns left.
|                   | Average | ara    | ben     | bul    | ces    | dan    | deu    | ell     | eng    | fas    | fin    | fra    | hbs    | heb     | hin    | hun    | ita    | jpn    | kor     | lit    | mar    | mkd    | msa    | nld    | nor    | pol    | por    | ron    | rus    | spa    | swe    | tgl    | tur    | ukr    | vie    | zho     |
|-------------------|---------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  76.08% | 99.36% | 100.00% | 73.71% | 71.26% | 51.25% | 63.60% | 100.00% | 74.52% | 97.13% | 88.35% | 48.13% |  8.56% |  98.44% | 80.18% | 88.91% | 38.29% | 99.87% | 100.00% | 92.87% | 83.30% | 80.28% | 64.03% | 62.82% | 38.70% | 96.67%

|                   | Average | ara    | ben     | bul    | ces    | dan    | deu    | ell     | eng    | fas    | fin    | fra    | hbs    | heb     | hin    | hun    | ita    | jpn    | kor     | lit    | mar    | mkd    | msa    | nld    | nor    | pol    | por    | ron    | rus    | spa    | swe    | tgl    | tur    | ukr    | vie    | zho     |
|-------------------|---------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  76.08% | 99.36% | 100.00% | 73.71% | 71.26% | 51.25% | 63.60% | 100.00% | 74.52% | 97.13% | 88.35% | 48.13% |  8.56% |  98.44% | 80.18% | 88.91% | 38.29% | 99.87% | 100.00% | 92.87% | 83.30% | 80.28% | 64.03% | 62.82% | 38.70% | 96.67% | 59.21% | 84.18% | 78.59% | 24.25% | 69.34% | 79.76% | 86.52% | 80.92% | 99.75% | 100.00% |
|   detector_ngram2 |  88.71% | 99.19% | 100.00% | 78.14% | 87.21% | 71.25% | 91.66% | 100.00% | 93.91% | 98.09% | 97.88% | 88.50% |  9.44% |  98.31% | 84.65% | 97.18% | 87.28% | 99.95% | 100.00% | 98.15% | 85.64% | 86.50% | 91.80% | 92.07% | 71.96% | 98.90% | 89.80% | 91.21% | 84.25% | 73.57% | 87.06% | 94.84% | 96.13% | 80.46% | 99.75% | 100.00% |
|   detector_ngram3 |  92.39% | 99.09% | 100.00% | 81.11% | 92.52% | 82.17% | 97.84% | 100.00% | 98.35% | 98.60% | 99.02% | 96.67% | 11.19% |  98.61% | 85.04% | 98.83% | 96.36% | 99.97% | 100.00% | 99.31% | 85.18% | 91.75% | 96.34% | 96.77% | 81.11% | 99.29% | 96.53% | 95.79% | 89.77% | 90.01% | 93.91% | 98.02% | 98.64% | 86.10% | 99.92% |  99.97% |
|   detector_ngram4 |  93.82% | 98.92% | 100.00% | 85.05% | 94.39% | 86.71% | 99.07% | 100.00% | 99.30% | 98.97% | 99.28% | 98.30% | 16.04% |  99.01% | 88.06% | 99.19% | 97.96% | 99.97% | 100.00% | 99.53% | 86.57% | 93.80% | 97.69% | 97.83% | 83.78% | 99.42% | 98.14% | 97.11% | 92.13% | 94.15% | 95.99% | 99.01% | 99.11% | 89.35% | 99.92% |  99.97% |
|   detector_ngram5 |  94.36% | 99.09% | 100.00% | 87.06% | 94.64% | 87.60% | 99.27% | 100.00% | 99.37% | 99.27% | 99.21% | 98.52% | 21.89% |  99.14% | 91.34% | 99.21% | 98.00% | 99.97% | 100.00% | 99.57% | 87.36% | 94.70% | 97.97% | 97.97% | 83.55% | 99.49% | 98.23% | 97.24% | 93.16% | 94.61% | 96.25% | 99.12% | 99.13% | 90.97% | 99.92% |  99.97% |
|      Languages.jl |  72.77% | 85.55% | 100.00% | 62.46% | 48.93% | 47.06% | 90.48% |  99.89% | 78.21% | 95.00% | 76.87% | 82.21% | 60.28% |  95.75% | 62.99% | 74.00% | 66.27% | 69.37% |  98.97% | 61.94% | 72.05% | 51.40% | 71.26% | 78.91% | 66.74% | 72.74% | 77.35% | 70.87% | 52.59% | 61.89% | 52.46% | 63.98% | 52.10% | 62.63% | 84.06% |  99.51% |
| LanguageDetect.jl |  80.72% | 93.61% | 100.00% | 64.47% | 71.64% | 53.66% | 82.15% | 100.00% | 74.55% | 93.53% | 90.31% | 76.88% | 25.98% | 100.00% | 91.60% | 86.49% | 69.26% | 99.84% |  99.48% | 80.60% | 86.74% | 74.55% | 85.82% | 65.21% | 52.72% | 92.47% | 70.31% | 83.49% | 78.27% | 60.45% | 70.36% | 90.83% | 90.27% | 71.78% | 99.49% |  98.45% |


In [20]:
showtable(TB, LANGS, threshold=0.5)

50 columns in total, 28 columns left.
|                   | Average | ara    | ben     | bul    | deu    | ell     | eng    | fas    | fin    | heb     | hin    | hun    | jpn    | kor     | lit    | mar    | mkd    | msa    | nld    | pol    | por    | ron    | rus    | swe    | tgl    | tur    | ukr    | vie    | zho     |
|-------------------|---------|--------|---------|--------|--------|---------|--------|--------|--------|---------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  85.08% | 99.36% | 100.00% | 73.71% | 63.60% | 100.00% | 74.52% | 97.13% | 88.35% |  98.44% | 80.18% | 88.91% | 99.87% | 100.00% | 92.87% | 83.30% | 80.28% | 64.03% | 62.82% | 96.67% | 59.21% | 84.18% | 78.59% | 69.34% | 79.76% | 86.52% | 80.92% | 99.75% | 100.00% |
|   detector_ngram2 |  93.41% | 99.19% | 100.00% | 78.14% | 91.66% | 100.00% | 93.91% | 98.09% 

|                   | Average | ara    | ben     | bul    | deu    | ell     | eng    | fas    | fin    | heb     | hin    | hun    | jpn    | kor     | lit    | mar    | mkd    | msa    | nld    | pol    | por    | ron    | rus    | swe    | tgl    | tur    | ukr    | vie    | zho     |
|-------------------|---------|--------|---------|--------|--------|---------|--------|--------|--------|---------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|
|   detector_ngram1 |  85.08% | 99.36% | 100.00% | 73.71% | 63.60% | 100.00% | 74.52% | 97.13% | 88.35% |  98.44% | 80.18% | 88.91% | 99.87% | 100.00% | 92.87% | 83.30% | 80.28% | 64.03% | 62.82% | 96.67% | 59.21% | 84.18% | 78.59% | 69.34% | 79.76% | 86.52% | 80.92% | 99.75% | 100.00% |
|   detector_ngram2 |  93.41% | 99.19% | 100.00% | 78.14% | 91.66% | 100.00% | 93.91% | 98.09% | 97.88% |  98.31% | 84.65% | 97.18% | 99.95% | 100.00% | 98.15% | 85.64% | 86.50% | 91.80% | 92.07% | 98.90% | 89.80% | 91.21% | 84.25% | 87.06% | 94.84% | 96.13% | 80.46% | 99.75% | 100.00% |
|   detector_ngram3 |  95.85% | 99.09% | 100.00% | 81.11% | 97.84% | 100.00% | 98.35% | 98.60% | 99.02% |  98.61% | 85.04% | 98.83% | 99.97% | 100.00% | 99.31% | 85.18% | 91.75% | 96.34% | 96.77% | 99.29% | 96.53% | 95.79% | 89.77% | 93.91% | 98.02% | 98.64% | 86.10% | 99.92% |  99.97% |
|   detector_ngram4 |  96.87% | 98.92% | 100.00% | 85.05% | 99.07% | 100.00% | 99.30% | 98.97% | 99.28% |  99.01% | 88.06% | 99.19% | 99.97% | 100.00% | 99.53% | 86.57% | 93.80% | 97.69% | 97.83% | 99.42% | 98.14% | 97.11% | 92.13% | 95.99% | 99.01% | 99.11% | 89.35% | 99.92% |  99.97% |
|   detector_ngram5 |  97.28% | 99.09% | 100.00% | 87.06% | 99.27% | 100.00% | 99.37% | 99.27% | 99.21% |  99.14% | 91.34% | 99.21% | 99.97% | 100.00% | 99.57% | 87.36% | 94.70% | 97.97% | 97.97% | 99.49% | 98.23% | 97.24% | 93.16% | 96.25% | 99.12% | 99.13% | 90.97% | 99.92% |  99.97% |
|      Languages.jl |  75.48% | 85.55% | 100.00% | 62.46% | 90.48% |  99.89% | 78.21% | 95.00% | 76.87% |  95.75% | 62.99% | 74.00% | 69.37% |  98.97% | 61.94% | 72.05% | 51.40% | 71.26% | 78.91% | 72.74% | 77.35% | 70.87% | 52.59% | 52.46% | 63.98% | 52.10% | 62.63% | 84.06% |  99.51% |
| LanguageDetect.jl |  86.24% | 93.61% | 100.00% | 64.47% | 82.15% | 100.00% | 74.55% | 93.53% | 90.31% | 100.00% | 91.60% | 86.49% | 99.84% |  99.48% | 80.60% | 86.74% | 74.55% | 85.82% | 65.21% | 92.47% | 70.31% | 83.49% | 78.27% | 70.36% | 90.83% | 90.27% | 71.78% | 99.49% |  98.45% |


## wikipedia

In [11]:
WB = benchmark(
"detector_ngram1"=>t->detector_lh(t, ngram=1),
"detector_ngram2"=>t->detector_lh(t, ngram=2),
"detector_ngram3"=>t->detector_lh(t, ngram=3),
"detector_ngram4"=>t->detector_lh(t, ngram=4),
"detector_ngram5"=>detector_lh,
"Languages.jl"=>detector_ls, 
"LanguageDetect.jl"=>detector_ld,
 dataset=WV, languages=LANGS)

detector_ngram1

  7.297311 seconds (18.98 M allocations: 1.299 GiB, 3.03% gc time, 1.31% compilation time)
detector_ngram2

 19.609008 seconds (37.25 M allocations: 2.512 GiB, 2.05% gc time, 0.37% compilation time)
detector_ngram3

 39.311345 seconds (55.49 M allocations: 3.933 GiB, 1.50% gc time, 0.18% compilation time)
detector_ngram4

 68.784380 seconds (73.71 M allocations: 5.453 GiB, 1.16% gc time, 0.08% compilation time)
detector_ngram5

 91.158265 seconds (91.91 M allocations: 7.130 GiB, 0.98% gc time, 0.09% compilation time)


Languages.jl 34.115749 seconds (104.71 M allocations: 2.425 GiB, 0.97% gc time, 0.16% compilation time)


LanguageDetect.jl 96.876576 seconds (988.71 M allocations: 35.064 GiB, 5.65% gc time, 0.08% compilation time)


7-element Vector{Any}:
   "detector_ngram1" => [0.995, 0.99, 1.0, 0.955, 0.99, 0.935, 0.875, 0.945, 1.0, 0.905  …  0.895, 0.965, 0.945, 0.995, 0.935, 0.985, 0.995, 0.975, 0.94, 1.0]
   "detector_ngram2" => [0.995, 0.99, 1.0, 0.96, 1.0, 0.96, 0.97, 0.96, 1.0, 0.99  …  0.985, 0.975, 0.995, 0.99, 0.98, 0.985, 1.0, 0.97, 0.96, 1.0]
   "detector_ngram3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.985, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.98, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]
   "detector_ngram4" => [1.0, 0.99, 1.0, 1.0, 1.0, 0.985, 0.995, 0.965, 1.0, 1.0  …  1.0, 0.99, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
   "detector_ngram5" => [1.0, 0.995, 1.0, 1.0, 1.0, 0.98, 0.995, 0.97, 1.0, 1.0  …  1.0, 0.99, 0.985, 0.99, 0.985, 0.985, 1.0, 0.965, 0.99, 0.995]
      "Languages.jl" => [0.99, 0.985, 0.99, 0.99, 0.0, 0.925, 0.885, 0.96, 0.965, 0.995  …  0.985, 0.0, 0.88, 0.0, 0.97, 0.925, 0.93, 0.745, 0.98, 0.76]
 "LanguageDetect.jl" => [0.995, 0.0, 1.0, 0.835, 0.785, 0.86, 0.58, 

In [12]:
showtable(WB, LANGS)

|                   | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs     | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-------------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|---------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------

|                   | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs     | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho     |
|-------------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|---------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|
|   detector_ngram1 |  89.75% |  99.50% | 99.00% | 100.00% |  95.50% |  99.00% | 93.50% | 87.50% | 94.50% | 100.00% |  90.50% |  92.00% |  99.50% | 97.50% |  92.50% | 86.00% |   5.50% | 100.00% | 86.50% |  99.00% | 70.50% |  1.50% | 96.50% | 82.50% | 91.50% | 97.50% | 100.00% | 99.00% | 94.00% |  99.00% | 92.00% | 94.50% | 96.00% | 86.00% | 83.00% | 47.50% |  99.00% | 92.00% | 89.00% |  99.50% | 96.50% |  89.50% | 96.50% | 94.50% | 99.50% | 93.50% | 98.50% |  99.50% | 97.50% | 94.00% | 100.00% |
|   detector_ngram2 |  94.90% |  99.50% | 99.00% | 100.00% |  96.00% | 100.00% | 96.00% | 97.00% | 96.00% | 100.00% |  99.00% |  97.00% |  99.50% | 99.00% |  99.00% | 97.50% |   7.50% | 100.00% | 89.50% | 100.00% | 84.50% | 71.00% | 98.00% | 97.00% | 96.00% | 98.50% | 100.00% | 99.00% | 95.00% | 100.00% | 91.50% | 97.00% | 99.00% | 98.00% | 95.00% | 85.50% | 100.00% | 98.00% | 90.50% |  99.50% | 96.50% |  98.50% | 97.50% | 99.50% | 99.00% | 98.00% | 98.50% | 100.00% | 97.00% | 96.00% | 100.00% |
|   detector_ngram3 |  95.93% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.00% |  10.50% | 100.00% | 89.50% | 100.00% | 88.50% | 84.50% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.00% | 97.00% | 99.00% | 98.50% | 96.50% | 91.00% | 100.00% | 99.50% | 90.00% |  99.50% | 96.50% | 100.00% | 98.50% | 98.00% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% |  99.50% |
|   detector_ngram4 |  97.63% | 100.00% | 99.00% | 100.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% |  66.50% | 100.00% | 90.00% |  99.50% | 93.00% | 93.00% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 95.00% | 98.00% | 99.00% | 99.00% | 97.50% | 93.50% |  99.50% | 99.50% | 91.00% | 100.00% | 96.50% | 100.00% | 99.00% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% |  99.50% |
|   detector_ngram5 |  98.22% | 100.00% | 99.50% | 100.00% | 100.00% | 100.00% | 98.00% | 99.50% | 97.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 99.00% |  88.00% | 100.00% | 90.50% |  99.50% | 94.50% | 93.50% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 97.50% | 99.00% | 99.00% | 99.00% | 97.50% | 94.50% |  99.50% | 99.50% | 91.00% | 100.00% | 96.50% | 100.00% | 99.00% | 98.50% | 99.00% | 98.50% | 98.50% | 100.00% | 96.50% | 99.00% |  99.50% |
|      Languages.jl |  71.84% |  99.00% | 98.50% |  99.00% |  99.00% |       - | 92.50% | 88.50% | 96.00% |  96.50% |  99.50% |  96.00% |  98.50% | 98.00% | 100.00% | 99.00% | 100.00% |  99.50% | 91.00% |  93.00% |      - |      - |      - | 98.50% |  2.50% |      - |  89.50% |      - |      - |  94.50% | 95.00% | 98.00% | 99.50% |      - | 94.50% | 95.50% |  90.50% | 94.00% | 81.50% |  97.50% |      - |  98.50% |      - | 88.00% |      - | 97.00% | 92.50% |  93.00% | 74.50% | 98.00% |  76.00% |
| LanguageDetect.jl |  65.33% |  99.50% |      - | 100.00% |  83.50% |  78.50% | 86.00% | 58.00% | 83.50% | 100.00% |  92.50% |       - |  99.00% | 93.50% |  85.50% |      - |   4.00% | 100.00% | 93.00% |  93.50% |      - |      - |      - | 91.00% | 96.00% |      - |  95.00% |      - |      - |  95.50% | 98.00% | 91.00% | 96.50% |      - | 77.50% | 58.50% |  96.00% | 78.00% | 70.00% |  91.00% | 78.50% |  73.00% | 95.00% | 70.50% |      - | 98.00% | 96.50% |  99.50% | 97.00% |      - |  74.50% |


In [13]:
showtable(WB, LANGS, threshold=0.)

50 columns in total, 35 columns left.
|                   | Average | ara     | ben     | bul     | ces    | dan    | deu    | ell     | eng     | fas     | fin    | fra     | hbs     | heb     | hin    | hun     | ita    | jpn    | kor     | lit     | mar    | mkd    | msa    | nld    | nor    | pol     | por    | ron    | rus     | spa     | swe    | tgl    | tur    | ukr     | vie    | zho     |
|-------------------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|--------|---------|---------|---------|--------|---------|--------|--------|---------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|---------|--------|---------|
|   detector_ngram1 |  90.86% |  99.50% | 100.00% |  95.50% | 93.50% | 87.50% | 94.50% | 100.00% |  90.50% |  99.50% | 97.50% |  92.50% |   5.50% | 100.00% | 86.50% |  99.00% | 82.50% | 91.50% | 100.00% |  99.00% | 92.00% | 94.50% | 9

|                   | Average | ara     | ben     | bul     | ces    | dan    | deu    | ell     | eng     | fas     | fin    | fra     | hbs     | heb     | hin    | hun     | ita    | jpn    | kor     | lit     | mar    | mkd    | msa    | nld    | nor    | pol     | por    | ron    | rus     | spa     | swe    | tgl    | tur    | ukr     | vie    | zho     |
|-------------------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|--------|---------|---------|---------|--------|---------|--------|--------|---------|---------|--------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|---------|--------|---------|
|   detector_ngram1 |  90.86% |  99.50% | 100.00% |  95.50% | 93.50% | 87.50% | 94.50% | 100.00% |  90.50% |  99.50% | 97.50% |  92.50% |   5.50% | 100.00% | 86.50% |  99.00% | 82.50% | 91.50% | 100.00% |  99.00% | 92.00% | 94.50% | 96.00% | 83.00% | 47.50% |  99.00% | 92.00% | 89.00% |  99.50% |  89.50% | 94.50% | 93.50% | 98.50% |  99.50% | 97.50% | 100.00% |
|   detector_ngram2 |  94.81% |  99.50% | 100.00% |  96.00% | 96.00% | 97.00% | 96.00% | 100.00% |  99.00% |  99.50% | 99.00% |  99.00% |   7.50% | 100.00% | 89.50% | 100.00% | 97.00% | 96.00% | 100.00% | 100.00% | 91.50% | 97.00% | 99.00% | 95.00% | 85.50% | 100.00% | 98.00% | 90.50% |  99.50% |  98.50% | 99.50% | 98.00% | 98.50% | 100.00% | 97.00% | 100.00% |
|   detector_ngram3 |  95.51% | 100.00% | 100.00% |  98.00% | 98.50% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% |  10.50% | 100.00% | 89.50% | 100.00% | 98.50% | 99.00% | 100.00% | 100.00% | 92.00% | 97.00% | 99.00% | 96.50% | 91.00% | 100.00% | 99.50% | 90.00% |  99.50% | 100.00% | 98.00% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|   detector_ngram4 |  97.49% | 100.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% |  66.50% | 100.00% | 90.00% |  99.50% | 99.00% | 99.00% | 100.00% | 100.00% | 95.00% | 98.00% | 99.00% | 97.50% | 93.50% |  99.50% | 99.50% | 91.00% | 100.00% | 100.00% | 98.50% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|   detector_ngram5 |  98.24% | 100.00% | 100.00% | 100.00% | 98.00% | 99.50% | 97.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% |  88.00% | 100.00% | 90.50% |  99.50% | 99.00% | 99.00% | 100.00% | 100.00% | 97.50% | 99.00% | 99.00% | 97.50% | 94.50% |  99.50% | 99.50% | 91.00% | 100.00% | 100.00% | 98.50% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|      Languages.jl |  91.44% |  99.00% |  99.00% |  99.00% | 92.50% | 88.50% | 96.00% |  96.50% |  99.50% |  98.50% | 98.00% | 100.00% | 100.00% |  99.50% | 91.00% |  93.00% | 98.50% |  2.50% |  89.50% |  94.50% | 95.00% | 98.00% | 99.50% | 94.50% | 95.50% |  90.50% | 94.00% | 81.50% |  97.50% |  98.50% | 88.00% | 97.00% | 92.50% |  93.00% | 74.50% |  76.00% |
| LanguageDetect.jl |  86.13% |  99.50% | 100.00% |  83.50% | 86.00% | 58.00% | 83.50% | 100.00% |  92.50% |  99.00% | 93.50% |  85.50% |   4.00% | 100.00% | 93.00% |  93.50% | 91.00% | 96.00% |  95.00% |  95.50% | 98.00% | 91.00% | 96.50% | 77.50% | 58.50% |  96.00% | 78.00% | 70.00% |  91.00% |  73.00% | 70.50% | 98.00% | 96.50% |  99.50% | 97.00% |  74.50% |


In [14]:
showtable(WB, LANGS, threshold=0.5)

50 columns in total, 32 columns left.
|                   | Average | ara     | ben     | bul     | ces    | dan    | deu    | ell     | eng     | fas     | fin    | fra     | heb     | hin    | hun     | ita    | kor     | lit     | mar    | mkd    | msa    | nld    | pol     | por    | ron    | rus     | spa     | swe    | tgl    | tur    | ukr     | vie    | zho     |
|-------------------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|--------|---------|---------|--------|---------|--------|---------|---------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|---------|--------|---------|
|   detector_ngram1 |  94.86% |  99.50% | 100.00% |  95.50% | 93.50% | 87.50% | 94.50% | 100.00% |  90.50% |  99.50% | 97.50% |  92.50% | 100.00% | 86.50% |  99.00% | 82.50% | 100.00% |  99.00% | 92.00% | 94.50% | 96.00% | 83.00% |  99.00% | 92.00% | 89.00% |  99.50% |  89.50% | 94.50% | 9

|                   | Average | ara     | ben     | bul     | ces    | dan    | deu    | ell     | eng     | fas     | fin    | fra     | heb     | hin    | hun     | ita    | kor     | lit     | mar    | mkd    | msa    | nld    | pol     | por    | ron    | rus     | spa     | swe    | tgl    | tur    | ukr     | vie    | zho     |
|-------------------|---------|---------|---------|---------|--------|--------|--------|---------|---------|---------|--------|---------|---------|--------|---------|--------|---------|---------|--------|--------|--------|--------|---------|--------|--------|---------|---------|--------|--------|--------|---------|--------|---------|
|   detector_ngram1 |  94.86% |  99.50% | 100.00% |  95.50% | 93.50% | 87.50% | 94.50% | 100.00% |  90.50% |  99.50% | 97.50% |  92.50% | 100.00% | 86.50% |  99.00% | 82.50% | 100.00% |  99.00% | 92.00% | 94.50% | 96.00% | 83.00% |  99.00% | 92.00% | 89.00% |  99.50% |  89.50% | 94.50% | 93.50% | 98.50% |  99.50% | 97.50% | 100.00% |
|   detector_ngram2 |  97.80% |  99.50% | 100.00% |  96.00% | 96.00% | 97.00% | 96.00% | 100.00% |  99.00% |  99.50% | 99.00% |  99.00% | 100.00% | 89.50% | 100.00% | 97.00% | 100.00% | 100.00% | 91.50% | 97.00% | 99.00% | 95.00% | 100.00% | 98.00% | 90.50% |  99.50% |  98.50% | 99.50% | 98.00% | 98.50% | 100.00% | 97.00% | 100.00% |
|   detector_ngram3 |  98.20% | 100.00% | 100.00% |  98.00% | 98.50% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 100.00% | 89.50% | 100.00% | 98.50% | 100.00% | 100.00% | 92.00% | 97.00% | 99.00% | 96.50% | 100.00% | 99.50% | 90.00% |  99.50% | 100.00% | 98.00% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|   detector_ngram4 |  98.53% | 100.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 100.00% | 90.00% |  99.50% | 99.00% | 100.00% | 100.00% | 95.00% | 98.00% | 99.00% | 97.50% |  99.50% | 99.50% | 91.00% | 100.00% | 100.00% | 98.50% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|   detector_ngram5 |  98.66% | 100.00% | 100.00% | 100.00% | 98.00% | 99.50% | 97.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 100.00% | 90.50% |  99.50% | 99.00% | 100.00% | 100.00% | 97.50% | 99.00% | 99.00% | 97.50% |  99.50% | 99.50% | 91.00% | 100.00% | 100.00% | 98.50% | 98.50% | 98.50% | 100.00% | 96.50% |  99.50% |
|      Languages.jl |  93.83% |  99.00% |  99.00% |  99.00% | 92.50% | 88.50% | 96.00% |  96.50% |  99.50% |  98.50% | 98.00% | 100.00% |  99.50% | 91.00% |  93.00% | 98.50% |  89.50% |  94.50% | 95.00% | 98.00% | 99.50% | 94.50% |  90.50% | 94.00% | 81.50% |  97.50% |  98.50% | 88.00% | 97.00% | 92.50% |  93.00% | 74.50% |  76.00% |
| LanguageDetect.jl |  89.25% |  99.50% | 100.00% |  83.50% | 86.00% | 58.00% | 83.50% | 100.00% |  92.50% |  99.00% | 93.50% |  85.50% | 100.00% | 93.00% |  93.50% | 91.00% |  95.00% |  95.50% | 98.00% | 91.00% | 96.50% | 77.50% |  96.00% | 78.00% | 70.00% |  91.00% |  73.00% | 70.50% | 98.00% | 96.50% |  99.50% | 97.00% |  74.50% |
